In [63]:
import pandas as pd

In [64]:
!pip install transformers


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [47]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
df = pd.read_csv('../csv_files/def_api.csv')
df

,classes,meainings
0,GetCurrentObject,GetCurrentObject function (wingdi.h) The GetCu...
1,ExitThread,Application.ExitThread Method Definition Names...
2,IsMenu,IsMenu function (winuser.h) Determines whether...
3,WSALookupServiceBeginA,WSALookupServiceBeginA function (winsock2.h) T...
4,DebugActiveProcess,DebugActiveProcess function (debugapi.h) Enabl...
...,...,...
661,SetFilePointer,Positioning a File Pointer When an application...
662,GetVolumeInformationA,GetVolumeInformationA function (fileapi.h) Ret...
663,GetTimeFormatA,GetTimeFormatA function (datetimeapi.h) Format...
664,102,MSSQLSERVER_102 Applies to: SQL Server Azure S...


In [49]:
import torch
from torch.utils.data import Dataset, DataLoader

class WordMeaningDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=64):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data[idx]["class"]
        meaning = self.data[idx]["meaning"]

        # Tokenize word and meaning together
        inputs = self.tokenizer(word, meaning, 
                                padding="max_length", 
                                truncation=True, 
                                max_length=self.max_length, 
                                return_tensors="pt")

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
        }
    # def __getitem__(self, idx):
    #     try:
    #         item = self.data[idx]  # Attempt to access the item at index `idx`
            
    #         # Debug: Print type of item to verify expected format
    #         print(f"Processing index: {idx}, Type: {type(item)}")

    #         # Ensure the item is a dictionary before accessing keys
    #         if not isinstance(item, dict):
    #             raise TypeError(f"Expected dictionary but got {type(item)} at index {idx}")

    #         word = item.get("class", "Unknown")  # Use .get() to avoid KeyErrors
    #         meaning = item.get("meaning", "No meaning found")

    #         return word, meaning

    #     except Exception as e:
    #         print(f"Error at index {idx}: {e}")  # Print error message
    #         print(f"Batch content at index {idx}: {self.data[idx]}")  # Show the problematic batch
    #         return None  # You can return None or handle it differently
    # def __getitem__(self, idx):
    #     try:
    #         item = self.data[idx]

    #         if not isinstance(item, dict):
    #             raise TypeError(f"Expected dict but got {type(item)} at index {idx}")

    #         return {
    #             "input_ids": torch.tensor(item["input_ids"], dtype=torch.long),
    #             "attention_mask": torch.tensor(item["attention_mask"], dtype=torch.long),
    #             "labels": torch.tensor(item["labels"], dtype=torch.long) if "labels" in item else None
    #         }

    #     except Exception as e:
    #         print(f"Error at index {idx}: {e}")
    #         print(f"Batch content at index {idx}: {self.data[idx]}")
    #         return None


In [50]:
# Example dataset
df = pd.read_csv('../csv_files/def_api.csv')
data = []
missed_data_points = []
for i,cla in enumerate(df['classes']):
  if isinstance(df['meainings'][i],str):
    data.append({'class':cla,'meaning':df['meainings'][i]})
  else:
    missed_data_points.append(cla)
print(data)

[{'class': 'GetCurrentObject', 'meaning': "GetCurrentObject function (wingdi.h) The GetCurrentObject function retrieves a handle to an object of the specified type that has been selected into the specified device context (DC). Syntax Parameters [in] hdc A handle to the DC. [in] type The object type to be queried. Remarks An application can use the GetCurrentObject and GetObject functions.GetColorAdjustment The GetColorAdjustment function retrieves the color adjustment values for the specified device context (DC). GetCurrentObject The GetCurrentObject function retrieves a handle to an object of the specified type that has been selected into the specified device context (DC). GetCurrentPositionEx The GetCurrentPositionEx function.GetColorAdjustment The Ge... GetColorSpace The GetColorSpace function retrieves the handle to the input color space from a specified device context. GetCurrentObject The GetCurrentObject function retrieves a handle to an object of the specified type that has bee

In [51]:
#tokenize dataset
dataset = WordMeaningDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [52]:
import torch.optim as optim
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define loss function
loss_fn = torch.nn.MSELoss()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Define dummy target labels (modify this for your task)
        target = torch.zeros_like(logits)

        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")


/Users/rajshekhar/Desktop/ci/project/venv/lib/python3.13/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens h

Epoch 1, Loss: 0.028870240325199033


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 2, Loss: 0.02517003678466852


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 3, Loss: 0.015999390550593778


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 4, Loss: 0.002965096343115608


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Epoch 5, Loss: 0.0007088557432303433


In [53]:
model.save_pretrained("../../bert_word_meaning_model")
tokenizer.save_pretrained("../../bert_word_meaning_model")

('../../bert_word_meaning_model/tokenizer_config.json',
 '../../bert_word_meaning_model/special_tokens_map.json',
 '../../bert_word_meaning_model/vocab.txt',
 '../../bert_word_meaning_model/added_tokens.json')

In [54]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("../../bert_word_meaning_model")
tokenizer = BertTokenizer.from_pretrained("../../bert_word_meaning_model")

In [55]:
tokenizer

BertTokenizer(name_or_path='../../bert_word_meaning_model', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [56]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class SentenceEncoder(nn.Module):
    def __init__(self, bert_model_name="../../bert_word_meaning_model", embed_dim=768, num_filters=128, kernel_size=3):
        super(SentenceEncoder, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)  # BERT as embedding layer
        self.conv1d = nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=kernel_size, padding=1)
        self.relu = nn.ReLU()
        self.max_pool = nn.AdaptiveMaxPool1d(1)  # Global max pooling

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():  # Freeze BERT during encoding
            bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        embeddings = bert_outputs.last_hidden_state  # (batch_size, seq_length, embed_dim)
        embeddings = embeddings.permute(0, 2, 1)  # Convert to (batch_size, embed_dim, seq_length) for CNN
        
        conv_output = self.conv1d(embeddings)  # Apply CNN
        conv_output = self.relu(conv_output)
        pooled_output = self.max_pool(conv_output).squeeze(-1)  # Get fixed-size sentence vector
        
        return pooled_output  # (batch_size, num_filters)

# Example usage
tokenizer = BertTokenizer.from_pretrained("../../bert_word_meaning_model")

def encode_sentence(sentence, max_length=128):
    tokens = tokenizer(sentence, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    return tokens["input_ids"], tokens["attention_mask"]

# Sample sentence with entity indicators
sentence = ""
input_ids, attention_mask = encode_sentence(sentence)

# Model forward pass
model = SentenceEncoder()
sentence_vector = model(input_ids, attention_mask)
print("Sentence representation shape:", sentence_vector.shape)  # Should be (1, num_filters)


Sentence representation shape: torch.Size([1, 128])


In [57]:
sentence_vector

tensor([[0.1344, 0.2120, 0.2986, 0.4345, 0.2239, 0.2020, 0.5086, 0.0000, 0.7391,
         0.2716, 0.1519, 0.2741, 0.2686, 0.2641, 0.2368, 0.4241, 0.4449, 0.1044,
         0.3728, 0.1470, 0.2787, 0.1519, 0.4734, 0.1454, 0.2682, 0.0887, 0.1843,
         0.3819, 0.5776, 0.3549, 0.1815, 0.2563, 0.2325, 0.3889, 0.1257, 0.0000,
         0.1501, 0.1083, 0.0426, 0.3675, 0.5241, 0.5040, 0.1433, 0.3029, 0.0957,
         0.4379, 0.2650, 0.3813, 0.0290, 0.0000, 0.5311, 0.3334, 0.1088, 0.0000,
         0.1023, 0.2391, 0.0872, 0.0636, 0.2218, 0.1404, 0.0923, 0.2292, 0.4235,
         0.4710, 0.1269, 0.1399, 0.4828, 0.2508, 0.3546, 0.0491, 0.2076, 0.3236,
         0.2509, 0.0000, 0.0917, 0.1222, 0.0881, 0.0612, 0.0665, 0.1394, 0.2598,
         0.0628, 0.2463, 0.3308, 0.2538, 0.1892, 0.1752, 0.3847, 0.4774, 0.0511,
         0.1905, 0.1490, 0.3886, 0.2563, 0.0000, 0.1475, 0.4788, 0.2357, 0.2184,
         0.1030, 0.3159, 0.0954, 0.0000, 0.4339, 0.0943, 0.2149, 0.5150, 0.0000,
         0.4279, 0.4177, 0.1

In [58]:
df_malware = pd.read_csv('../csv_files/after_data.csv')

In [59]:
df_malware

,Unnamed: 0,class,sha256,api_1,api_2,api_3,api_4,api_5,api_6,api_7,...,api_96,api_97,api_98,api_99,api_100,api_101,api_102,api_103,api_104,api_105
0,0,worm,009a83236c600fd7ac034973f064284cec62f86631fe96...,GetSystemDirectoryA,IsDBCSLeadByte,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,GlobalAddAtomW,lstrcpynW,...,LoadLibraryExW,DuplicateHandle,DeviceIoControl,SwitchToThread,WSACleanup,FreeLibrary,VirtualQueryEx,ResetEvent,VirtualFreeEx,UnregisterClassW
1,1,worm,191828200aa2bd78f3548fb6a47407b05905efee6e83e0...,GetSystemDirectoryA,IsDBCSLeadByte,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,GlobalAddAtomW,lstrcpynW,...,LoadLibraryExW,DuplicateHandle,DeviceIoControl,SwitchToThread,WSACleanup,FreeLibrary,VirtualQueryEx,ResetEvent,VirtualFreeEx,UnregisterClassW
2,2,worm,1a1937fe8a401ca27be4514acc4f811442d8201abbad2a...,GetSystemDirectoryA,IsDBCSLeadByte,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,GlobalAddAtomW,lstrcpynW,...,LoadLibraryExW,DuplicateHandle,DeviceIoControl,SwitchToThread,WSACleanup,FreeLibrary,VirtualQueryEx,ResetEvent,VirtualFreeEx,UnregisterClassW
3,3,worm,21bbf357ac6d8347c68ec65fb8283a5f1185f699186959...,GetSystemDirectoryA,IsDBCSLeadByte,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,GlobalAddAtomW,lstrcpynW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,worm,39563933eebcb3b09b189000e1bda255b39739792fc136...,GetSystemDirectoryA,IsDBCSLeadByte,LocalAlloc,CreateSemaphoreW,CreateSemaphoreA,GlobalAddAtomW,lstrcpynW,...,LoadLibraryExW,DuplicateHandle,DeviceIoControl,SwitchToThread,WSACleanup,FreeLibrary,VirtualQueryEx,ResetEvent,VirtualFreeEx,UnregisterClassW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1589,1589,trojan,1fa088e4bfce3a269cd69fad5dc632fa977e6741a73f04...,GlobalFindAtomW,GetModuleHandleW,DisableThreadLibraryCalls,GetVersionExW,GetVersionExA,GetModuleHandleA,GetProcAddress,...,GetProcAddress,GetWindowThreadProcessId,GetCurrentProcessId,LocalFree,CloseHandle,CoTaskMemFree,NaN,NaN,NaN,NaN
1590,1590,worm,1fad69fad4aac201134b9600e1596d327097c38046e2ff...,GetSystemDirectoryA,GetVersionExW,IsDBCSLeadByte,CompareStringW,DisableThreadLibraryCalls,CreateEventW,lstrcpyW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1591,1591,virus,1fb2ce95769c2372560f34b00398f3681949f494bfdf1c...,DisableThreadLibraryCalls,GetVersionExW,GetVersionExA,GetModuleHandleW,GetModuleHandleA,GetProcAddress,LocalAlloc,...,VirtualQueryEx,GetModuleFileNameW,WriteFile,UnregisterClassW,GetModuleHandleW,GetProcAddress,GetWindowThreadProcessId,GetCurrentProcessId,LocalFree,CloseHandle
1592,1592,trojan,1fb8e85deb472dadf965916020ac9ccec0104d882eb332...,VirtualAllocEx,GetStartupInfoA,GetStdHandle,GetFileType,LockResource,GetCommandLineA,GetEnvironmentStringsW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
api_seq = [df_malware['api_1'][0],df_malware['api_2'][0],df_malware['api_3'][0],df_malware['api_4'][0],df_malware['api_5'][0],df_malware['api_6'][0]]
print(api_seq)

['GetSystemDirectoryA', 'IsDBCSLeadByte', 'LocalAlloc', 'CreateSemaphoreW', 'CreateSemaphoreA', 'GlobalAddAtomW']


In [61]:
api_encodings = {}

In [62]:
for i in api_seq:
    input_ids, attention_mask = encode_sentence(i)
    model = SentenceEncoder()
    sentence_vector = model(input_ids, attention_mask)
    api_encodings[i] = sentence_vector
    print(f"{i} : {sentence_vector}") 

GetSystemDirectoryA : tensor([[0.4095, 0.3133, 0.2994, 0.1804, 0.0665, 0.2500, 0.3580, 0.0000, 0.1933,
         0.4644, 0.2058, 0.0448, 0.1827, 0.2373, 0.0000, 0.6709, 0.2961, 0.5023,
         0.1209, 0.1299, 0.5623, 0.2689, 0.1453, 0.2367, 0.1283, 0.1277, 0.1783,
         0.0773, 0.0000, 0.2634, 0.0225, 0.1328, 0.0867, 0.2591, 0.3791, 0.2022,
         0.4378, 0.4529, 0.3004, 0.0044, 0.1386, 0.2388, 0.3604, 0.0840, 0.0000,
         0.3119, 0.2547, 0.2214, 0.5245, 0.3255, 0.0815, 0.3345, 0.2104, 0.1682,
         0.2719, 0.4134, 0.0000, 0.1337, 0.0000, 0.5796, 0.2846, 0.1970, 0.0413,
         0.3006, 0.1215, 0.0000, 0.3601, 0.1570, 0.3169, 0.2431, 0.3239, 0.0688,
         0.1052, 0.1194, 0.1920, 0.1220, 0.2072, 0.1768, 0.0602, 0.0352, 0.6909,
         0.0476, 0.4752, 0.3378, 0.1181, 0.0792, 0.4809, 0.2471, 0.3054, 0.1936,
         0.3735, 0.2061, 0.2521, 0.2464, 0.5390, 0.3867, 0.4592, 0.0000, 0.4327,
         0.2265, 0.1419, 0.0806, 0.2323, 0.3702, 0.0000, 0.0260, 0.1689, 0.0840,
      

In [44]:
api_encodings

{'GetSystemDirectoryA': tensor([[0.1791, 0.0897, 0.4013, 0.0913, 0.3151, 0.1763, 0.2815, 0.0365, 0.4221,
          0.4213, 0.3429, 0.1986, 0.0061, 0.3363, 0.3899, 0.2745, 0.4443, 0.0000,
          0.2221, 0.7003, 0.3193, 0.1772, 0.1659, 0.4850, 0.2074, 0.4410, 0.0454,
          0.7320, 0.1208, 0.1634, 0.2454, 0.1600, 0.5079, 0.1913, 0.4262, 0.6356,
          0.0000, 0.6557, 0.5203, 0.1328, 0.2936, 0.0309, 0.3477, 0.2330, 0.1146,
          0.4291, 0.3936, 0.0000, 0.1235, 0.0241, 0.2558, 0.0000, 0.6882, 0.4521,
          0.0866, 0.0553, 0.0099, 0.3571, 0.3162, 0.0000, 0.2845, 0.2768, 0.3188,
          0.3708, 0.0000, 0.3231, 0.5861, 0.7029, 0.1408, 0.3252, 0.0000, 0.3491,
          0.0000, 0.1961, 0.4351, 0.2685, 0.2551, 0.1990, 0.2989, 0.1004, 0.1775,
          0.3583, 0.3025, 0.3262, 0.0527, 0.2519, 0.3159, 0.1675, 0.5502, 0.6872,
          0.0000, 0.3727, 0.4296, 0.3348, 0.4670, 0.0000, 0.0130, 0.3149, 0.5536,
          0.2844, 0.4304, 0.2721, 0.2217, 0.0530, 0.0927, 0.3196, 0.6662, 0

In [65]:
api_encodings2 = {}
for i in api_seq:
    input_ids, attention_mask = encode_sentence(i,max_length=21)
    model = SentenceEncoder()
    sentence_vector = model(input_ids, attention_mask)
    api_encodings2[i] = sentence_vector
    print(f"{i} : {sentence_vector}") 

GetSystemDirectoryA : tensor([[0.1702, 0.0948, 0.3632, 0.2980, 0.0000, 0.1676, 0.0815, 0.1435, 0.3647,
         0.1241, 0.2442, 0.2072, 0.0000, 0.2428, 0.0000, 0.3141, 0.0204, 0.4215,
         0.3480, 0.3281, 0.1594, 0.2672, 0.2343, 0.2005, 0.0000, 0.3530, 0.2317,
         0.3703, 0.4326, 0.1178, 0.0000, 0.0000, 0.0000, 0.0755, 0.1190, 0.0511,
         0.1724, 0.1242, 0.3329, 0.4753, 0.2282, 0.3451, 0.2723, 0.1274, 0.4604,
         0.3927, 0.2075, 0.0703, 0.2507, 0.1654, 0.2375, 0.3210, 0.1090, 0.0813,
         0.1255, 0.0000, 0.2225, 0.5715, 0.0000, 0.0296, 0.4663, 0.1082, 0.2556,
         0.3168, 0.2251, 0.2397, 0.0000, 0.3537, 0.3356, 0.5153, 0.1166, 0.3317,
         0.1883, 0.3022, 0.4325, 0.2898, 0.1416, 0.1108, 0.6034, 0.0949, 0.2483,
         0.0749, 0.2407, 0.0296, 0.3713, 0.3276, 0.0170, 0.5291, 0.3401, 0.1795,
         0.0620, 0.2000, 0.0897, 0.3333, 0.3453, 0.0495, 0.3450, 0.3761, 0.3235,
         0.3772, 0.3910, 0.3988, 0.4989, 0.1951, 0.5296, 0.0798, 0.0000, 0.1925,
      

In [ ]:
api_encodings2

In [39]:
sentence_vector

tensor([[0.4907, 0.4222, 0.0000, 0.3999, 0.4758, 0.3901, 0.0000, 0.5541, 0.0505,
         0.6262, 0.1223, 0.3879, 0.4151, 0.2004, 0.2518, 0.2718, 0.2575, 0.0865,
         0.4626, 0.1429, 0.0889, 0.4190, 0.4350, 0.4156, 0.2874, 0.0000, 0.5432,
         0.2532, 0.0166, 0.1277, 0.2530, 0.4031, 0.0128, 0.4125, 0.3263, 0.3251,
         0.2090, 0.4125, 0.2497, 0.5074, 0.2928, 0.3767, 0.7107, 0.0459, 0.4249,
         0.2378, 0.3416, 0.2791, 0.2498, 0.3475, 0.5838, 0.5298, 0.3248, 0.4576,
         0.2675, 0.1830, 0.0000, 0.5551, 0.0805, 0.3740, 0.3836, 0.2753, 0.5755,
         0.7983, 0.1613, 0.3692, 0.0826, 0.3429, 0.2091, 0.4378, 0.3228, 0.3954,
         0.1943, 0.4002, 0.2969, 0.2972, 0.2551, 0.2124, 0.3899, 0.0845, 0.7423,
         0.1856, 0.5603, 0.1537, 0.1061, 0.1910, 0.3674, 0.3680, 0.1135, 0.1624,
         0.3041, 0.1200, 0.3008, 0.0000, 0.2311, 0.0000, 0.6468, 0.1770, 0.0000,
         0.4106, 0.4127, 0.1586, 0.2548, 0.3382, 0.2424, 0.1789, 0.4270, 0.4180,
         0.3731, 0.2871, 0.0

In [36]:
tokens = []
for i in api_seq:
    tokens.append(tokenizer.tokenize(i))
print(tokens)

[['gets', '##yst', '##em', '##di', '##re', '##ctor', '##ya'], ['is', '##db', '##cs', '##lea', '##db', '##yte'], ['local', '##all', '##oc'], ['creates', '##ema', '##ph', '##ore', '##w'], ['creates', '##ema', '##ph', '##ore', '##a'], ['global', '##ad', '##da', '##tom', '##w']]


In [ ]:
import json
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset

class FewRelDataset(Dataset):
    """
    Dataset class for FewRel dataset
    """
    def __init__(self, data_path, tokenizer, max_length=64):
        """
        Initialize FewRel dataset
        
        Args:
            data_path: Path to the FewRel JSON file
            tokenizer: BERT tokenizer
            max_length: Maximum sequence length
        """
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Load data
        with open(data_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
            
        # Get relation list
        self.rel_list = list(self.data.keys())
        
        # Pre-process data
        self.preprocess_data()
    
    def preprocess_data(self):
        """
        Pre-process the data for easy sampling
        """
        self.processed_data = {}
        
        for rel in self.rel_list:
            self.processed_data[rel] = []
            
            for instance in self.data[rel]:
                tokens = instance['tokens']
                text = ' '.join(tokens)
                
                # Get entity positions
                head_pos = instance['h'][2][0]
                tail_pos = instance['t'][2][0]
                
                # Convert token positions to character positions
                head_start = len(' '.join(tokens[:head_pos[0]]))
                if head_pos[0] > 0:
                    head_start += 1  # Add space
                head_end = head_start + len(' '.join(tokens[head_pos[0]:head_pos[-1]+1]))
                
                tail_start = len(' '.join(tokens[:tail_pos[0]]))
                if tail_pos[0] > 0:
                    tail_start += 1  # Add space
                tail_end = tail_start + len(' '.join(tokens[tail_pos[0]:tail_pos[-1]+1]))
                
                self.processed_data[rel].append({
                    'text': text,
                    'head_entity': ' '.join(tokens[head_pos[0]:head_pos[-1]+1]),
                    'tail_entity': ' '.join(tokens[tail_pos[0]:tail_pos[-1]+1]),
                    'head_pos': (head_start, head_end),
                    'tail_pos': (tail_start, tail_end)
                })
    
    def sample_task(self, n_way, k_shot, n_query):
        """
        Sample a few-shot task
        
        Args:
            n_way: Number of relations (N)
            k_shot: Number of support instances per relation (K)
            n_query: Number of query instances per relation
        
        Returns:
            task: Dictionary containing the sampled task
        """
        # Sample N relations
        sampled_rels = random.sample(self.rel_list, n_way)
        
        support_sentences = []
        support_entity_pairs = []
        support_labels = []
        query_sentences = []
        query_entity_pairs = []
        query_labels = []
        
        for i, rel in enumerate(sampled_rels):
            instances = self.processed_data[rel]
            # Sample K+Q instances without replacement
            sampled_instances = random.sample(instances, k_shot + n_query)
            
            # Support set
            for j in range(k_shot):
                instance = sampled_instances[j]
                support_sentences.append(instance['text'])
                support_entity_pairs.append((
                    instance['head_pos'][0],
                    instance['head_pos'][1],
                    instance['tail_pos'][0],
                    instance['tail_pos'][1]
                ))
                support_labels.append(i)
            
            # Query set
            for j in range(k_shot, k_shot + n_query):
                instance = sampled_instances[j]
                query_sentences.append(instance['text'])
                query_entity_pairs.append((
                    instance['head_pos'][0],
                    instance['head_pos'][1],
                    instance['tail_pos'][0],
                    instance['tail_pos'][1]
                ))
                query_labels.append(i)
        
        # Return the task
        return {
            'support_sentences': support_sentences,
            'support_entity_pairs': support_entity_pairs,
            'support_labels': support_labels,
            'query_sentences': query_sentences,
            'query_entity_pairs': query_entity_pairs,
            'query_labels': query_labels,
            'relations': sampled_rels
        }
    
    def __len__(self):
        """
        Number of relations in the dataset
        """
        return len(self.rel_list)
    
    def __getitem__(self, idx):
        """
        Get all instances for a specific relation
        """
        rel = self.rel_list[idx]
        return self.processed_data[rel]

def get_data_loaders(train_path, val_path, tokenizer, max_length=64):
    """
    Get FewRel data loaders for training and validation
    
    Args:
        train_path: Path to training data
        val_path: Path to validation data
        tokenizer: BERT tokenizer
        max_length: Maximum sequence length
    
    Returns:
        train_dataset: Training dataset
        val_dataset: Validation dataset
    """
    train_dataset = FewRelDataset(train_path, tokenizer, max_length)
    val_dataset = FewRelDataset(val_path, tokenizer, max_length)
    
    return train_dataset, val_dataset

# Function to update the training function with the FewRel dataset
def train_model_with_fewrel(model, train_dataset, val_dataset, n_way=5, k_shot=5, 
                          learning_rate=2e-5, num_iterations=10000, eval_interval=100):
    """
    Train the model on few-shot relation classification task using FewRel dataset
    
    Args:
        model: FewShotRelationClassification model
        train_dataset: FewRel training dataset
        val_dataset: FewRel validation dataset
        n_way: Number of relations (classes)
        k_shot: Number of instances per relation
        learning_rate: Learning rate
        num_iterations: Number of training iterations
        eval_interval: Evaluation interval
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.model.to(device)
    
    optimizer = torch.optim.Adam(model.model.parameters(), lr=learning_rate)
    
    best_accuracy = 0.0
    
    for iteration in range(num_iterations):
        # Sample task for this iteration
        task = train_dataset.sample_task(n_way, k_shot, n_query=k_shot)
        
        # Move data to device
        support_inputs, support_masks = model.prepare_batch(
            task['support_sentences'], 
            task['support_entity_pairs']
        )
        query_inputs, query_masks = model.prepare_batch(
            task['query_sentences'], 
            task['query_entity_pairs']
        )
        
        support_inputs = support_inputs.to(device)
        support_masks = support_masks.to(device)
        query_inputs = query_inputs.to(device)
        query_masks = query_masks.to(device)
        
        query_labels = torch.tensor(task['query_labels']).to(device)
        
        # Training step
        model.model.train()
        optimizer.zero_grad()
        
        # Forward pass
        logits = model.model(support_inputs, support_masks, query_inputs, query_masks, n_way, k_shot)
        
        # Calculate loss
        loss = torch.nn.functional.cross_entropy(logits, query_labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Evaluate on validation set every eval_interval iterations
        if (iteration + 1) % eval_interval == 0:
            val_accuracy = evaluate_model_fewrel(model, val_dataset, n_way, k_shot, device)
            
            print(f"Iteration {iteration + 1}/{num_iterations}, Loss: {loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")
            
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                # Save model weights
                torch.save(model.model.state_dict(), f"best_few_shot_model_{n_way}way_{k_shot}shot.pt")
    
    return best_accuracy

def evaluate_model_fewrel(model, val_dataset, n_way=5, k_shot=5, device=None, num_tasks=100):
    """
    Evaluate the model on validation data using FewRel dataset
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model.model.eval()
    
    total_correct = 0
    total_instances = 0
    
    with torch.no_grad():
        for _ in range(num_tasks):
            # Sample task
            task = val_dataset.sample_task(n_way, k_shot, n_query=k_shot)
            
            # Move data to device
            support_inputs, support_masks = model.prepare_batch(
                task['support_sentences'], 
                task['support_entity_pairs']
            )
            query_inputs, query_masks = model.prepare_batch(
                task['query_sentences'], 
                task['query_entity_pairs']
            )
            
            support_inputs = support_inputs.to(device)
            support_masks = support_masks.to(device)
            query_inputs = query_inputs.to(device)
            query_masks = query_masks.to(device)
            
            # Forward pass
            logits = model.model(support_inputs, support_masks, query_inputs, query_masks, n_way, k_shot)
            
            # Get predictions
            _, predictions = torch.max(logits, dim=1)
            
            # Compare with ground truth
            correct = (predictions.cpu().numpy() == np.array(task['query_labels'])).sum()
            total_correct += correct
            total_instances += len(predictions)
    
    accuracy = total_correct / total_instances
    return accuracy

# Example usage with FewRel dataset
def main_fewrel():
    from transformers import BertTokenizer
    
    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    
    # Initialize model
    model = FewShotRelationClassification(
        pretrained_bert_name="bert-base-uncased",
        hidden_size=768,
        max_len=64,
        num_heads=12
    )
    
    # Load data
    train_path = "data/train.json"  # Path to FewRel training data
    val_path = "data/val.json"      # Path to FewRel validation data
    
    train_dataset, val_dataset = get_data_loaders(train_path, val_path, tokenizer, max_length=64)
    
    # Train model
    best_accuracy = train_model_with_fewrel(
        model,
        train_dataset,
        val_dataset,
        n_way=5,
        k_shot=5,
        learning_rate=2e-5,
        num_iterations=10000,
        eval_interval=100
    )
    
    print(f"Best validation accuracy: {best_accuracy:.4f}")

if __name__ == "__main__":
    main_fewrel()

DATA PROCESSING

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

class ApiSequenceDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=64):
        """
        Dataset for API call sequences
        
        Args:
            data_path: Path to the CSV file containing API sequences
            tokenizer: BERT tokenizer
            max_length: Maximum sequence length (for padding/truncation)
        """
        self.data = pd.read_csv(data_path)
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Extract API sequence columns (all columns starting with 'api_seq_')
        self.api_cols = [col for col in self.data.columns if col.startswith('api_seq_')]
        
        # Create class labels mapping
        self.classes = self.data['class_name'].unique()
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Extract API sequence for this sample (handling variable length)
        api_sequence = []
        for col in self.api_cols:
            if pd.notna(row[col]):  # Only include non-NaN values
                api_sequence.append(str(row[col]))
        
        # Convert class name to index
        label = self.class_to_idx[row['class_name']]
        
        # Tokenize the API sequence
        # We join API calls with spaces to treat them as words
        api_sequence_str = " ".join(api_sequence)
        
        # Tokenize using BERT tokenizer
        encoding = self.tokenizer(
            api_sequence_str,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Remove batch dimension added by tokenizer
        for key in encoding:
            encoding[key] = encoding[key].squeeze(0)
        
        # Add sample metadata
        encoding['sha256'] = row['sha256']
        encoding['label'] = torch.tensor(label)
        encoding['sequence_length'] = min(len(api_sequence), self.max_length)
        
        return encoding


Collacation Function

In [ ]:
def collate_api_sequences(batch):
    """
    Collate function to handle variable length API sequences in a batch
    """
    # Get the maximum sequence length in this batch
    max_len = max([sample['sequence_length'] for sample in batch])
    
    # Initialize batch dictionaries
    batch_encoding = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': [],
        'sha256': [],
        'label': [],
        'sequence_length': []
    }
    
    # Collect samples
    for sample in batch:
        # Truncate or pad to the max length in this batch
        # We already padded to max_length in the dataset, but we might need to adjust
        batch_encoding['input_ids'].append(sample['input_ids'][:max_len])
        batch_encoding['attention_mask'].append(sample['attention_mask'][:max_len])
        if 'token_type_ids' in sample:
            batch_encoding['token_type_ids'].append(sample['token_type_ids'][:max_len])
        
        # Add metadata
        batch_encoding['sha256'].append(sample['sha256'])
        batch_encoding['label'].append(sample['label'])
        batch_encoding['sequence_length'].append(sample['sequence_length'])
    
    # Convert lists to tensors
    batch_encoding['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        batch_encoding['input_ids'], batch_first=True, padding_value=0
    )
    batch_encoding['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        batch_encoding['attention_mask'], batch_first=True, padding_value=0
    )
    if batch_encoding['token_type_ids']:
        batch_encoding['token_type_ids'] = torch.nn.utils.rnn.pad_sequence(
            batch_encoding['token_type_ids'], batch_first=True, padding_value=0
        )
    
    batch_encoding['label'] = torch.stack(batch_encoding['label'])
    batch_encoding['sequence_length'] = torch.tensor(batch_encoding['sequence_length'])
    
    return batch_encoding


Modified Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class Proto_CATT_BERT_API(nn.Module):
    """
    Context Attention-based Prototypical Networks with BERT for API Sequence Classification
    Modified to handle variable-length API call sequences
    """
    def __init__(self, bert_model='bert-base-uncased', hidden_size=768, conv_window_size=3, num_heads=12, num_classes=2):
        super(Proto_CATT_BERT_API, self).__init__()
        
        # BERT embedding layer
        self.bert = BertModel.from_pretrained(bert_model)
        self.hidden_size = hidden_size
        
        # CNN encoding layer
        self.conv = nn.Conv1d(hidden_size, hidden_size, kernel_size=conv_window_size, 
                             padding=(conv_window_size-1)//2)
        
        # Number of attention heads
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads
        
        # Classification layer (if not using prototypical networks)
        self.classifier = nn.Linear(hidden_size, num_classes)
        
        assert self.head_dim * num_heads == hidden_size, "hidden_size must be divisible by num_heads"
    
    def sentence_encoder(self, input_ids, attention_mask=None, token_type_ids=None):
        """
        Encode a sentence using BERT and CNN
        """
        # Get BERT embeddings
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        word_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
        
        # Apply CNN for feature extraction
        # Transpose for conv1d which expects (batch_size, channels, seq_len)
        word_embeddings = word_embeddings.transpose(1, 2)  # (batch_size, hidden_size, seq_len)
        conv_output = self.conv(word_embeddings)  # (batch_size, hidden_size, seq_len)
        
        # Apply max pooling over sequence length
        # This handles variable length by taking the maximum feature value across the sequence
        sentence_embedding = F.max_pool1d(conv_output, kernel_size=conv_output.size(2))
        sentence_embedding = sentence_embedding.squeeze(2)  # (batch_size, hidden_size)
        
        return sentence_embedding
    
    def context_attention(self, support_set):
        """
        Apply context attention to the support set
        
        Implementation of equation: softmax(S*S^T/sqrt(d_w))*S with multi-head attention
        """
        batch_size, num_instances, _ = support_set.size()
        
        # Split into multiple heads
        support_set_heads = support_set.view(batch_size, num_instances, self.num_heads, self.head_dim)
        support_set_heads = support_set_heads.permute(0, 2, 1, 3)  # (batch_size, num_heads, num_instances, head_dim)
        
        # Calculate attention for each head
        attention_scores = torch.matmul(support_set_heads, support_set_heads.transpose(-2, -1))
        attention_scores = attention_scores / (self.head_dim ** 0.5)  # Scale by sqrt(d_w)
        attention_weights = F.softmax(attention_scores, dim=-1)
        
        # Apply attention weights
        weighted_heads = torch.matmul(attention_weights, support_set_heads)
        
        # Combine heads
        weighted_heads = weighted_heads.permute(0, 2, 1, 3)  # (batch_size, num_instances, num_heads, head_dim)
        weighted_support = weighted_heads.reshape(batch_size, num_instances, self.hidden_size)
        
        return weighted_support
    
    def forward_prototypical(self, query_inputs, support_inputs, n_way, k_shot):
        """
        Forward pass for few-shot learning using prototypical networks
        """
        # Process query instances
        query_embedding = self.sentence_encoder(
            query_inputs['input_ids'], 
            attention_mask=query_inputs.get('attention_mask', None), 
            token_type_ids=query_inputs.get('token_type_ids', None)
        )  # (batch_size, hidden_size)
        
        # Process support set instances
        batch_size = support_inputs['input_ids'].size(0)
        support_instance_count = support_inputs['input_ids'].size(1)  # n_way * k_shot
        
        # Reshape for processing each instance individually
        flat_input_ids = support_inputs['input_ids'].view(-1, support_inputs['input_ids'].size(-1))
        flat_attention_mask = None
        if 'attention_mask' in support_inputs:
            flat_attention_mask = support_inputs['attention_mask'].view(-1, support_inputs['attention_mask'].size(-1))
        flat_token_type_ids = None
        if 'token_type_ids' in support_inputs:
            flat_token_type_ids = support_inputs['token_type_ids'].view(-1, support_inputs['token_type_ids'].size(-1))
        
        # Encode all support instances
        flat_support_embeddings = self.sentence_encoder(
            flat_input_ids, 
            attention_mask=flat_attention_mask, 
            token_type_ids=flat_token_type_ids
        )  # (batch_size * support_instance_count, hidden_size)
        
        # Reshape back
        support_embeddings = flat_support_embeddings.view(batch_size, support_instance_count, -1)
        
        # Apply context attention to highlight important instances
        weighted_support = self.context_attention(support_embeddings)
        
        # Reshape support set for n-way k-shot
        support_embeddings = weighted_support.view(batch_size, n_way, k_shot, -1)
        
        # Calculate prototypes by averaging the k instances for each class
        prototypes = support_embeddings.mean(dim=2)  # (batch_size, n_way, hidden_size)
        
        # Calculate Euclidean distances between query and prototypes
        query_expanded = query_embedding.unsqueeze(1)  # (batch_size, 1, hidden_size)
        distances = torch.sum((query_expanded - prototypes) ** 2, dim=2)  # (batch_size, n_way)
        
        # Convert distances to logits (negative distances)
        logits = -distances
        
        return logits
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        """
        Standard forward pass for classification (non-few-shot)
        """
        # Get sentence embeddings
        embeddings = self.sentence_encoder(
            input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids
        )
        
        # Get logits through the classifier
        logits = self.classifier(embeddings)
        
        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            return loss, logits
        
        return logits


Training Code

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import random
import os
from tqdm import tqdm

# Set random seed for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Add special tokens for API calls if needed
special_tokens = ['<PAD>', '<UNK>']  # Add any API-specific tokens here
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# Load datasets
train_dataset = ApiSequenceDataset(
    data_path='path_to_your_train_data.csv',
    tokenizer=tokenizer,
    max_length=64  # Adjust based on your API sequence lengths
)

val_dataset = ApiSequenceDataset(
    data_path='path_to_your_validation_data.csv',
    tokenizer=tokenizer,
    max_length=64
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=8,  # Adjust based on your GPU memory
    shuffle=True,
    collate_fn=collate_api_sequences
)

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_api_sequences
)

# Initialize the model
num_classes = len(train_dataset.classes)
model = Proto_CATT_BERT_API(
    bert_model='bert-base-uncased',
    hidden_size=768,
    conv_window_size=3,
    num_heads=12,
    num_classes=num_classes
)

# Resize model embeddings if special tokens were added
model.bert.resize_token_embeddings(len(tokenizer))

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * 10  # 10 epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Training loop
def train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs=10):
    best_accuracy = 0
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase
        model.train()
        total_loss = 0
        
        for batch in tqdm(train_loader, desc="Training"):
            # Move batch to device
            batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            loss, _ = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                token_type_ids=batch.get('token_type_ids', None),
                labels=batch['label']
            )
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Update parameters
            optimizer.step()
            scheduler.step()
            
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")
        
        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                # Move batch to device
                batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
                
                # Forward pass
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    token_type_ids=batch.get('token_type_ids', None)
                )
                
                # Get predictions
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                labels = batch['label'].cpu().numpy()
                
                all_preds.extend(preds)
                all_labels.extend(labels)
        
        # Calculate accuracy
        accuracy = accuracy_score(all_labels, all_preds)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        # Print classification report
        print("\nClassification Report:")
        print(classification_report(
            all_labels, 
            all_preds, 
            target_names=[train_dataset.classes[i] for i in range(num_classes)]
        ))
        
        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_api_model.pt')
            print(f"Model saved with accuracy: {best_accuracy:.4f}")
    
    return model

# Train the model
trained_model = train_model(model, train_loader, val_loader, optimizer, scheduler)


In [ ]:
def prepare_few_shot_batch(dataset, n_way, k_shot, q_query):
    """
    Prepare a few-shot batch from the dataset
    
    Args:
        dataset: The full dataset
        n_way: Number of classes in the task
        k_shot: Number of support examples per class
        q_query: Number of query examples per class
        
    Returns:
        support_inputs, query_inputs, query_labels
    """
    # Randomly select n_way classes
    selected_classes = random.sample(range(len(dataset.classes)), n_way)
    
    # Initialize support and query sets
    support_inputs = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': []
    }
    query_inputs = {
        'input_ids': [],
        'attention_mask': [],
        'token_type_ids': []
    }
    query_labels = []
    
    # For each selected class
    for i, class_idx in enumerate(selected_classes):
        # Find all samples of this class
        class_samples = [
            idx for idx in range(len(dataset)) 
            if dataset[idx]['label'] == class_idx
        ]
        
        # Randomly select k_shot + q_query samples
        selected_samples = random.sample(
            class_samples, 
            min(k_shot + q_query, len(class_samples))
        )
        
        # Split into support and query
        support_samples = selected_samples[:k_shot]
        query_samples = selected_samples[k_shot:k_shot + q_query]
        
        # Add to support set
        for idx in support_samples:
            sample = dataset[idx]
            support_inputs['input_ids'].append(sample['input_ids'])
            support_inputs['attention_mask'].append(sample['attention_mask'])
            if 'token_type_ids' in sample:
                support_inputs['token_type_ids'].append(sample['token_type_ids'])
        
        # Add to query set
        for idx in query_samples:
            sample = dataset[idx]
            query_inputs['input_ids'].append(sample['input_ids'])
            query_inputs['attention_mask'].append(sample['attention_mask'])
            if 'token_type_ids' in sample:
                query_inputs['token_type_ids'].append(sample['token_type_ids'])
            
            # Query label is the index in the selected_classes list (0 to n_way-1)
            query_labels.append(i)
    
    # Stack tensors
    for key in support_inputs:
        if support_inputs[key]:
            support_inputs[key] = torch.stack(support_inputs[key])
    
    for key in query_inputs:
        if query_inputs[key]:
            query_inputs[key] = torch.stack(query_inputs[key])
    
    query_labels = torch.tensor(query_labels)
    
    return support_inputs, query_inputs, query_labels

# Evaluate few-shot learning
def evaluate_few_shot(model, dataset, n_way=5, k_shot=5, q_query=5, num_tasks=100):
    """
    Evaluate the model on few-shot learning tasks
    """
    model.eval()
    accuracies = []
    
    for _ in tqdm(range(num_tasks), desc=f"Evaluating {n_way}-way {k_shot}-shot tasks"):
        # Prepare a few-shot batch
        support_inputs, query_inputs, query_labels = prepare_few_shot_batch(
            dataset, n_way, k_shot, q_query
        )
        
        # Move to device
        support_inputs = {k: v.to(device) for k, v in support_inputs.items()}
        query_inputs = {k: v.to(device) for k, v in query_inputs.items()}
        query_labels = query_labels.to(device)
        
        # Forward pass in few-shot mode
        with torch.no_grad():
            logits = model.forward_prototypical(
                query_inputs, support_inputs, n_way, k_shot
            )
            
            # Get predictions
            preds = torch.argmax(logits, dim=1)
            
            # Calculate accuracy
            accuracy = (preds == query_labels).float().mean().item()
            accuracies.append(accuracy)
    
    # Calculate mean and standard deviation
    mean_acc = np.mean(accuracies)
    std_acc = np.std(accuracies)
    
    print(f"{n_way}-way {k_shot}-shot accuracy: {mean_acc:.4f} ± {std_acc:.4f}")
    
    return mean_acc, std_acc

# Example usage
test_dataset = ApiSequenceDataset(
    data_path='path_to_your_test_data.csv',
    tokenizer=tokenizer,
    max_length=64
)

# Load the best model
model.load_state_dict(torch.load('best_api_model.pt'))
model.to(device)

# Evaluate on few-shot tasks
evaluate_few_shot(model, test_dataset, n_way=5, k_shot=5)
